In [66]:
import numpy as np
import pandas as pd
import os
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from isotopde_data_v5 import isotopes


In [67]:
#Covert text files into csv
#convert Absorption Cross Section Data
df_abs = pd.read_csv('Absorption_Cross_Section_Data.txt', delim_whitespace=True)
df_abs.to_csv('Absorption_Cross_Section_Data.csv', index=False)

#convert Capture Cross Section Data
df_capture = pd.read_csv('Capture_Cross_Section_Data.txt', delim_whitespace=True)
df_capture.to_csv('Capture_Cross_Section_Data.csv', index=False)

#convert n2n Cross Section Data
df_n2n = pd.read_csv('n2n_Cross_Section_Data.txt', delim_whitespace=True)
df_n2n.to_csv('n2n_Cross_Section_Data.csv', index=False)

#convert Neutron Energy Groups Data
# Read the file with multi-word headers using proper header rows and column names
df_neutron_energy = pd.read_csv(
    'Neutron_Energy_Groups.txt',
    delim_whitespace=True,
    header=None,
    skiprows=1,
    names=[
        'Energy Group',
        'Lower Boundary (eV)',
        'Upper Boundary (eV)'
    ]
)
df_neutron_energy.to_csv('Neutron_Energy_Groups.csv', index=False)


In [68]:
#Create a function to load cross-section data from CSV files
def load_cross_section_data(file_path):
    df = pd.read_csv(file_path)
    df.set_index(df.columns[0], inplace=True)  # Use the first column as index (energy groups)
    return df.to_dict(orient='list')  # Each isotope maps to a list of values over energy groups


# Load cross-section data
absorption_data = load_cross_section_data('Absorption_Cross_Section_Data.csv')
capture_data = load_cross_section_data('Capture_Cross_Section_Data.csv')
n2n_data = load_cross_section_data('n2n_Cross_Section_Data.csv')

#Build a dictionary for each reaction type
absorption_data = {isotope: data for isotope, data in absorption_data.items() if isotope in isotopes}
capture_data = {isotope: data for isotope, data in capture_data.items() if isotope
in isotopes}
n2n_data = {isotope: data for isotope, data in n2n_data.items() if isotope in isotopes}
# Create a dictionary to hold all cross-section data
cross_section_data = {
    'absorption': absorption_data,
    'capture': capture_data,
    'n2n': n2n_data
}
#print("absorption_data:", absorption_data)
#print("capture_data:", capture_data)
#print("n2n_data:", n2n_data)

In [69]:
# Integrate cross-section data into the isotopes dictionary
for iso in isotopes:
    # Add capture (n, gamma)
    if iso in cross_section_data['capture']:
        isotopes[iso]['n_gamma'] = cross_section_data['capture'][iso]
    else:
        isotopes[iso]['n_gamma'] = 0.0  # Or a vector if using multi-group

    # Add n,2n
    if iso in cross_section_data['n2n']:
        isotopes[iso]['n_2n'] = cross_section_data['n2n'][iso]
    else:
        isotopes[iso]['n_2n'] = 0.0

    # Add absorption (as fallback for fission or additional capture)
    if iso in cross_section_data['absorption']:
        if isotopes[iso].get('n_fission', 0.0) == 0.0:
            isotopes[iso]['n_fission'] = cross_section_data['absorption'][iso]
    else:
        if 'n_fission' not in isotopes[iso]:
            isotopes[iso]['n_fission'] = 0.0


In [71]:
# Load 175-group flux (no header, one column)
flux = pd.read_csv('Flux.txt', header=None).squeeze().values

# Normalize flux if desired (optional, but helps numerical stability)
flux = flux / np.sum(flux)

# Perform flux-weighted averaging for each reaction type
for iso in isotopes:
    # Handle (n,γ)
    if 'capture' in isotopes[iso]:
        capture = np.array(isotopes[iso]['capture'])
        isotopes[iso]['n_gamma'] = float(np.sum(capture * flux))
    else:
        isotopes[iso]['n_gamma'] = 0.0

    # Handle (n,2n)
    if 'n2n' in isotopes[iso]:
        n2n = np.array(isotopes[iso]['n2n'])
        isotopes[iso]['n_2n'] = float(np.sum(n2n * flux))
    else:
        isotopes[iso]['n_2n'] = 0.0

    # Handle absorption fallback for fission
    if 'absorption' in isotopes[iso]:
        absorption = np.array(isotopes[iso]['absorption'])
        avg_abs = float(np.sum(absorption * flux))

        # Only use absorption if n_gamma and n_fission are zero
        if isotopes[iso].get('n_gamma', 0.0) == 0.0 and isotopes[iso].get('n_fission', 0.0) == 0.0:
            isotopes[iso]['n_fission'] = avg_abs
    else:
        if 'n_fission' not in isotopes[iso]:
            isotopes[iso]['n_fission'] = 0.0
isotope_list = list(isotopes.keys())

# Map isotope to index for the ODE system
index_map = {iso: i for i, iso in enumerate(isotope_list)}
# Load materials (no header)
materials_df = pd.read_csv('Materials.txt', delim_whitespace=True, header=None)

# Define the system of ODEs
phi = 1.0  # Neutron flux (can be adjusted based on reactor conditions)
# This is a constant for simplicity, but can be made time-dependent if needed
def odes(t, y):
    dydt = np.zeros_like(y)
    for iso, props in isotopes.items():
        i = index_map[iso]

        # Loss from neutron reactions and decay
        # n_fission may be a list (energy-dependent), so sum if needed
        n_gamma = props.get('n_gamma', 0.0)
        n_2n = props.get('n_2n', 0.0)
        n_fission = props.get('n_fission', 0.0)
        if isinstance(n_fission, list) or isinstance(n_fission, np.ndarray):
            n_fission = float(np.sum(np.array(n_fission) * flux))
        loss = (n_gamma + n_2n + n_fission) * phi * y[i] + props['decay'] * y[i]

        # Gain from parent decays or reactions
        prod = 0.0
        for parent, reaction in props['prod']:
            j = index_map[parent]
            if reaction == 'n_gamma':
                parent_ngamma = isotopes[parent].get('n_gamma', 0.0)
                prod += parent_ngamma * phi * y[j]
            elif reaction == 'n_2n':
                parent_n2n = isotopes[parent].get('n_2n', 0.0)
                prod += parent_n2n * phi * y[j]
            elif reaction == 'n_fission':
                parent_nfission = isotopes[parent].get('n_fission', 0.0)
                if isinstance(parent_nfission, list) or isinstance(parent_nfission, np.ndarray):
                    parent_nfission = float(np.sum(np.array(parent_nfission) * flux))
                prod += parent_nfission * phi * y[j]
            elif reaction == 'decay':
                prod += isotopes[parent]['decay'] * y[j]

        dydt[i] = prod - loss
    return dydt


# Initialize concentrations
N0 = np.zeros(len(isotope_list))
for _, row in materials_df.iterrows():
    iso = row[0]
    amount = row[1]
    if iso in index_map:
        N0[index_map[iso]] = amount
    else:
        print(f"Warning: Isotope '{iso}' not found in the isotope model.")
        
# Ask user how many years to simulate
years = float(input("Enter burnup duration in years (e.g., 10): "))
T_end = years * 365.25 * 24 * 3600  # convert years to seconds
t_eval = np.linspace(0, T_end, 1000)



# Solve the system
sol = solve_ivp(odes, [0, T_end], N0, t_eval=t_eval, method='BDF')
"""
# Plot the results
plt.figure(figsize=(12, 7))
for i, iso in enumerate(isotope_list):
    plt.plot(sol.t / (365.25 * 24 * 3600), sol.y[i], label=iso)

plt.xlabel('Time (years)')
plt.ylabel('Relative Concentration')
plt.title('Isotope Production in U-238 Burnup Chain')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
"""
# Output the final concentrations
for i, iso in enumerate(isotope_list):
    val = sol.y[i][-1]
    if val != 0:
        print(f"{iso}: {val:.4e}")


U234: 4.9948e+19
U235: 2.8207e-58
U238: 4.5954e-56
